In [7]:
import os
#import random
#import pandas as pd
import numpy as np
#import glob
#import matplotlib.pyplot as plt
#plt.style.use("ggplot")
#%matplotlib inline

from sklearn.model_selection import KFold, train_test_split
import cv2

from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from keras.optimizers import Adam
from tensorflow.keras.models import Model, load_model

from keras_unet_collection import models #Keras unet collection
#import tensorflow as tf
from PIL import Image
from datetime import datetime

## DL Trans Unet without k-fold

In [ ]:
#define directory where images and masks are located on local disk
image_directory = 'D:/UniBas/Bachelorarbeit/Img_masks/DeepACSA_images_RF/insert_images/'
mask_directory = 'D:/UniBas/Bachelorarbeit/Img_masks/DeepACSA_masks_RF/insert_masks/'


#define the properties and empty list for resized images and masks
SIZE = 256
image_dataset = []
mask_dataset = []


#define custom function
def IoU(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    union = K.sum(y_true,-1) + K.sum(y_pred,-1) - intersection
    iou = (intersection + smooth) / ( union + smooth)
    return iou

#enumerate and resize images/masks
images = os.listdir(image_directory)
for i, image_name in enumerate(images):    #enumerate method adds a counter and returns the enumerate object
    if (image_name.split('.')[1] == 'tif'):
        #print(image_directory+image_name)
        image = cv2.imread(image_directory+image_name, 1)
        image = Image.fromarray(image)
        image = image.resize((SIZE, SIZE))
        image_dataset.append(np.array(image))


masks = os.listdir(mask_directory)
for i, image_name in enumerate(masks):
    if (image_name.split('.')[1] == 'tif'):
        image = cv2.imread(mask_directory+image_name, 0)
        image = Image.fromarray(image)
        image = image.resize((SIZE, SIZE))
        mask_dataset.append(np.array(image))

#define some hyperparameters
num_labels = 1  #Binary classificaion (missmatch with literature/code examples!)
batch_size = 2  #keep it smaller than 3
epochs = 60

#normalize images
image_dataset = np.array(image_dataset)/255.
#do not normalize masks, just rescale to 0 to 1.
mask_dataset = np.expand_dims((np.array(mask_dataset)),3) /255.

#define callback function
callbacks = [
  EarlyStopping(patience=8, verbose=1),
  ReduceLROnPlateau(factor=0.1, patience=10, min_lr=0.00001, verbose=1),
  ModelCheckpoint('A_modelTransV1.h5', verbose=1, save_best_only=True, save_weights_only=False), # Give the model a name (the .h5 part)
  CSVLogger('A_modelTransV1.csv', separator=',', append=False)]

#define the model architecture
#model requires depth >= 2
model = models.transunet_2d((256, 256, 3), filter_num=[64, 128, 256, 512, 1024], 
                          n_labels=num_labels, stack_num_down=2, stack_num_up=2, 
                          embed_dim=768, num_mlp=3072, num_heads=12, num_transformer=12, 
                          activation='ReLU', mlp_activation='GELU', output_activation='Sigmoid', #output activation from Softmax to Sigmoid
                          batch_norm=True, pool=True, unpool=False, name='transunet')
                          #batchnorm to true, unpool to false
#compile the model
model.compile(loss='binary_crossentropy', optimizer=Adam(lr = 1e-3), 
            metrics=['accuracy', IoU])

#split dataset into training and validation set
X_train, X_test, y_train, y_test = train_test_split(image_dataset, mask_dataset, test_size = 0.10, random_state = 0)

#start time meauserment
start2 = datetime.now() 

# Fit data to model
resunet_history = model.fit(X_train, y_train, 
                    verbose=1,
                    batch_size = batch_size,
                    validation_data=(X_test, y_test), 
                    shuffle=False,
                    epochs=epochs,
                    callbacks=callbacks)

#stop time measurement and print the execution time
stop2 = datetime.now() 
execution_time = stop2-start2
print("R2-Unet fitting time is: ", execution_time)

## Dl Trans with K-fold-cross validation (k = 5)

In [3]:
#define directory where images and masks are located
image_directory = 'D:/UniBas/Bachelorarbeit/Img_masks/DeepACSA_images_GMGL/insert_images/'
mask_directory = 'D:/UniBas/Bachelorarbeit/Img_masks/DeepACSA_masks_GMGL/insert_masks/'

#define the properties and empty list for resized images and masks
SIZE = 256
image_dataset = []
mask_dataset = []

#define custom function
def IoU(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    union = K.sum(y_true,-1) + K.sum(y_pred,-1) - intersection
    iou = (intersection + smooth) / ( union + smooth)
    return iou

#enumerate and resize images/masks
images = os.listdir(image_directory)
for i, image_name in enumerate(images):    #enumerate method adds a counter and returns the enumerate object
    if (image_name.split('.')[1] == 'tif'):
        #print(image_directory+image_name)
        image = cv2.imread(image_directory+image_name, 1)
        image = Image.fromarray(image)
        image = image.resize((SIZE, SIZE))
        image_dataset.append(np.array(image))

masks = os.listdir(mask_directory)
for i, image_name in enumerate(masks):
    if (image_name.split('.')[1] == 'tif'):
        image = cv2.imread(mask_directory+image_name, 0)
        image = Image.fromarray(image)
        image = image.resize((SIZE, SIZE))
        mask_dataset.append(np.array(image))

#define some hyperparameters
num_labels = 1  #Binary classificaion (missmatch on coding examples! 1 or 2 for binary classification)
batch_size = 2  #keep it smaller than 3
epochs = 60
num_folds = 5   #define the number of folds (usually 5-10 folds)

#Normalize images
image_dataset = np.array(image_dataset)/255.
#do not normalize masks, just rescale to 0 to 1.
mask_dataset = np.expand_dims((np.array(mask_dataset)),3) /255.

#define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=False)

#define per-fold score containers 
acc_per_fold = []
loss_per_fold = []
IoU_per_fold = []

#K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(image_dataset, mask_dataset):
  callbacks = [
    EarlyStopping(patience=8, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=10, min_lr=0.00001, verbose=1),
    ModelCheckpoint(f'B1GC-Kfoldno{fold_no}-transunet.h5', verbose=1, save_best_only=True, save_weights_only=False), # Give the model a name (the .h5 part)
    CSVLogger(f'B1GC-Kfoldno{fold_no}-transunet.csv', separator=',', append=False)]
  
  #define the model architecture
  model = models.transunet_2d((256, 256, 3), filter_num=[64, 128, 256, 512, 1024], 
                          n_labels=num_labels, stack_num_down=2, stack_num_up=2, 
                          embed_dim=768, num_mlp=3072, num_heads=12, num_transformer=12, 
                          activation='ReLU', mlp_activation='GELU', output_activation='Sigmoid', #output activation from Softmax to Sigmoid
                          batch_norm=True, pool=True, unpool=False, name='transunet')
                          #batchnorm to true, unpool to false

  #compile the model
  model.compile(loss='binary_crossentropy', optimizer=Adam(lr = 1e-3), 
              metrics=['accuracy', IoU])

  #generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  #fit model to data
  transunet_history = model.fit(image_dataset[train], mask_dataset[train], 
                    verbose=1,
                    batch_size = batch_size,
                    validation_data=(image_dataset[test], mask_dataset[test]), 
                    shuffle=False,
                    epochs=epochs,
                    callbacks=callbacks)

  #append evaluation values for every fold to a list
  acc_per_fold.append(model.evaluate(image_dataset[test], mask_dataset[test])[1])
  loss_per_fold.append(model.evaluate(image_dataset[test], mask_dataset[test])[0])
  IoU_per_fold.append(model.evaluate(image_dataset[test], mask_dataset[test])[2])

  #increase fold number
  fold_no += 1

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]} - IoU: {IoU_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print(f'> IoU: {np.mean(IoU_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Training for fold 1 ...
25/25 [==============================] - 167s 6s/step - loss: 0.5308 - accuracy: 0.7957 - IoU: 0.7609 - val_loss: 29112.0312 - val_accuracy: 0.0870 - val_IoU: 0.5458

Epoch 00001: val_loss improved from inf to 29112.03125, saving model to K-foldno1-transunet-V1-VL-256.h5
------------------------------------------------------------------------
Training for fold 2 ...
25/25 [==============================] - 164s 6s/step - loss: 0.4302 - accuracy: 0.8539 - IoU: 0.8031 - val_loss: 22961.9434 - val_accuracy: 0.0870 - val_IoU: 0.5459

Epoch 00001: val_loss improved from inf to 22961.94336, saving model to K-foldno2-transunet-V1-VL-256.h5
------------------------------------------------------------------------
Training for fold 3 ...
26/26 [==============================] - 189s 7s/step - loss: 0.4942 - accuracy: 0.8033 - IoU: 0.7731 - val_loss: 18681.1582 - val_accuracy: 0.0851 - val_IoU: 0.5446

c:\ProgramData\Anaconda3\envs\DeepACSA\lib\site-packages\numpy\core\fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\ProgramData\Anaconda3\envs\DeepACSA\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
c:\ProgramData\Anaconda3\envs\DeepACSA\lib\site-packages\numpy\core\_methods.py:233: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\ProgramData\Anaconda3\envs\DeepACSA\lib\site-packages\numpy\core\_methods.py:194: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(
c:\ProgramData\Anaconda3\envs\DeepACSA\lib\site-packages\numpy\core\_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
